In [7]:
import time
import scrapy
import random
import pymongo
import pandas as pd
from retry import retry
import requests
from bs4 import BeautifulSoup
from pymongo import TEXT
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait


In [8]:
# url = "http://free-proxy.cz/en/proxylist/country/MY/all/speed/all"
# page = requests.get(url)
# bs = BeautifulSoup(page.content)
# table = bs.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="Table1") 
# rows = table.findAll(lambda tag: tag.name=='tr')
# bs.find("table", {"id":"proxy_list"})

In [9]:
# bs.find("table")

In [15]:
def browse(url, proxy = None, wait_factor = 1):
    print("wait_factor:", wait_factor)
    #     url = 'https://httpbin.org/ip'
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    if proxy is not None:
        chrome_options.add_argument('--proxy-server=%s' % proxy)
    chrome_options.add_argument('--no-sandbox')
    
    capabilities = DesiredCapabilities.CHROME.copy()
    capabilities['acceptSslCerts'] = True 
    capabilities['acceptInsecureCerts'] = True
    
    driver = webdriver.Chrome(executable_path='/home/ubuntu/chromedriver',chrome_options=chrome_options,desired_capabilities=capabilities)
    driver.implicitly_wait(10*wait_factor) #seconds
    driver.get(url)
    delay = 20*wait_factor 
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, '__next')))
        print("Page is ready!")
    except TimeoutException:
        print("Loading took too much time!")
   
    page_source = driver.page_source
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.close()
    return page_source, soup
    
class CrawlMKListing():
    MONGO_URI = "localhost:27017"
    MONGO_DB = "news"
    MONGO_COLLECTION = "malaysiakini_v1_test1"
    DOMAIN = "https://www.malaysiakini.com"
    NEWS_LISTING_DOMAIN = "https://www.malaysiakini.com/stories/covid19"
    PROXY_LIST = [
        '45.117.228.65:4145',
        '210.48.204.118:13629',
        '103.220.5.254:4145',
        '103.220.6.106:4145',
        '45.117.228.81:4145',
        '121.122.50.157:4145',
        '60.53.199.121:8080',
        '175.143.51.221:4145',
    ]
    PROXY_LIST = ["socks4://" + j for j in PROXY_LIST]
    
#     PROXY_LIST = [
#                   "socks4://120.50.56.137:40553","socks4://121.122.50.157:4145", 
#                   "socks4://1.9.167.36:60489","socks4://1.9.111.145:4145",
#                   "socks4://45.117.228.153:4145","socks4://45.117.228.97:4145",
#                   "socks4://103.220.6.254:4145"
#                  ]
    START_URL = NEWS_LISTING_DOMAIN
    
    def __init__(self):
        self.wait_factor = 1
        self.CURRENT_PAGE_URL = self.START_URL
        continue_crawl = True
        while continue_crawl:
            next_page = self.crawl()
            continue_crawl = self.next_crawl(next_page)
    
    @retry(tries=5, delay=2)
    def crawl(self):
        try:
            CURRENT_PAGE_URL = self.CURRENT_PAGE_URL
            self.init_mongo()
            random.shuffle(self.PROXY_LIST)
            proxy = self.PROXY_LIST[0]
            print("Using proxy:", proxy, "Crawling:", CURRENT_PAGE_URL)
            page_source, soup = browse(url = CURRENT_PAGE_URL, proxy = proxy, wait_factor = self.wait_factor)
            _check = self.check(soup)
            # if _check is not True, retry the whole crawl for this page
            if _check: 
                to_insert, next_page = self.parse(soup)
                to_insert_2 = self.prevent_duplicate(to_insert)
                if len(to_insert_2) > 0:
                    self.coll.insert_many(to_insert_2)
            else:
                print("False check.")
                raise Exception("False check")
        except Exception as err:
            print(err)
            self.wait_factor += 1
            raise Exception("Error: crawl.")
        
        if self.wait_factor >= 3:
            self.wait_factor -= 1
        return next_page
            
    def next_crawl(self, next_page):
        if next_page is not None:
            self.CURRENT_PAGE_URL = next_page
            return True
        else:
            print("End crawl...")
            return False
            
            
    def init_mongo(self):
        client = pymongo.MongoClient(self.MONGO_URI)
        self.coll = client[self.MONGO_DB][self.MONGO_COLLECTION]

    def check(self, soup):
        try:
            x = soup.find("title").get_text()
        except:
            return False
        if x is None:
            return False
        if x.find("Access denied") >= 0:
            return False
        return True

    def parse(self, soup):
        DOMAIN = self.DOMAIN
        NEWS_LISTING_DOMAIN = self.NEWS_LISTING_DOMAIN
        
        x = soup.find("div", "news").find_all("a")[:-1]
        titles = [j.find("h3").getText() for j in x]
        urls = [j.get("href") for j in x]
        urls = [DOMAIN + url.replace(DOMAIN, "") for url in urls]
        df = pd.DataFrame(dict(title = titles, url = urls)) #.sample(30)
        if df.shape[0] == 0:
            raise Exception("shape 0")
        to_insert = df.to_dict(orient="records")
        
        try:
            next_button = soup.find("div", "news").find_all("a")[-1].get("href")
            next_page = NEWS_LISTING_DOMAIN + next_button
        except:
            next_page = None

        return to_insert, next_page
    
    def prevent_duplicate(self, to_insert):
        url_list = [j.get("url") for j in to_insert]
        exist_url_list = [j.get("url") for j in list(self.coll.find({"url": {"$in": url_list}}, {"url":1}))]
        to_insert_2 = [j for j in to_insert if j.get("url") not in exist_url_list]
        return to_insert_2

In [16]:
%%time
C = CrawlMKListing()
C.crawl()


Using proxy: socks4://60.53.199.121:8080 Crawling: https://www.malaysiakini.com/stories/covid19
wait_factor: 1


/home/ubuntu/anaconda3/envs/covid/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


Loading took too much time!
False check.
False check
Using proxy: socks4://45.117.228.81:4145 Crawling: https://www.malaysiakini.com/stories/covid19
wait_factor: 2
Page is ready!
Using proxy: socks4://45.117.228.65:4145 Crawling: https://www.malaysiakini.com/stories/covid19?page=1
wait_factor: 2
Page is ready!
'NoneType' object has no attribute 'getText'
Using proxy: socks4://103.220.6.106:4145 Crawling: https://www.malaysiakini.com/stories/covid19?page=1
wait_factor: 3
Loading took too much time!
False check.
False check
Using proxy: socks4://175.143.51.221:4145 Crawling: https://www.malaysiakini.com/stories/covid19?page=1
wait_factor: 4
Loading took too much time!
False check.
False check
Using proxy: socks4://175.143.51.221:4145 Crawling: https://www.malaysiakini.com/stories/covid19?page=1
wait_factor: 5


KeyboardInterrupt: 

In [12]:
# %%time
# # p = "https://202.187.91.243:3128"
# p = "socks4://210.48.204.118:13629"
# browse("https://www.malaysiakini.com/stories/covid19?page=1", p,1)